# Demo 01: Echogram fu
### `themachinethatgoesping` tutorial series

Here we show how to create echograms from acoustic data and how to do magic with them

In [9]:
%matplotlib widget

import numpy as np
import themachinethatgoesping as theping
from matplotlib import pyplot as plt
from tqdm.auto import tqdm

In [15]:
# data folder
folder = '/home/data/turbeams/TURBEAMS_data_crunshing/campaigns/PLUME_0/'

# find all Kongsberg files in the list of folders
files = sorted(theping.echosounders.index_functions.find_files(folder,['.raw']))
cache_files = theping.echosounders.index_functions.get_index_paths(files)

# create the file handler
fileHandler = theping.echosounders.simradraw.SimradRawFileHandler(files, cache_files)


Found 50 files
indexing files ⠐ 99% :00s<00m:00s] [Found: 1881429 datagrams in 50 files (23769MB)]                                                
Initializing ping interface ⠠ 99% :10s<00m:00s] [Done]                                              


# Prepare pings for echogram processing

In [29]:
# get pings that contain WCD
pings = theping.pingprocessing.filter_pings.by_features(fileHandler.get_pings(),['watercolumn.amplitudes'])

# split pings by channle
pings_by_channel = theping.pingprocessing.split_pings.by_channel_id(pings)

In [30]:
for k,p in pings_by_channel.items():
    print(k,len(p))

WBT 400051-15 ES120-7C_ES 106599
WBT 400038-15 ES18_ES 106599
WBT 400052-15 ES200-7C_ES 106599
WBT 400040-15 ES38-7_ES 106599
WBT 400058-15 ES333-7C_ES 106599
WBT 400050-15 ES70-7C_ES 106599


In [34]:
keys = pings_by_channel.keys()
keys = [
    'WBT 400058-15 ES333-7C_ES', 
    'WBT 400052-15 ES200-7C_ES', 
    'WBT 400051-15 ES120-7C_ES', 
    'WBT 400050-15 ES70-7C_ES',
    'WBT 400040-15 ES38-7_ES', 
    'WBT 400038-15 ES18_ES', 
]

# create echograms from the data

In [35]:
echogram_data = {}

for k in tqdm(keys):
    echogram_data[k] = theping.pingprocessing.watercolumn.echograms.EchogramBuilder.from_pings(
        pings_by_channel[k],
        pss=theping.echosounders.pingtools.PingSampleSelector(),
        wci_value = 'sv/av/pv/rv',
        linear_mean=True,
        no_navigation=False,
        apply_pss_to_bottom=False,
        force_angle=None,
        depth_stack=False,
        verbose=False)

  0%|          | 0/6 [00:00<?, ?it/s]

In [36]:
# set axis
for k,e in tqdm(echogram_data.items()):
    e.set_x_axis_date_time()
    e.set_y_axis_depth()

  0%|          | 0/6 [00:00<?, ?it/s]

# Display echogram using echogram viewer

In [37]:
echogramviewer = theping.pingprocessing.widgets.EchogramViewer(echogram_data)